# "音色无关转录"模型评估
不用log处理，直接用能量谱

## 生成转录结果并保存
经过process后每个文件夹里的文件只有后缀不同，且后缀为"npy" "wav" "mid"。wav采样率已经是22050Hz。

In [ ]:
import os
from instrument_agnostic_eval_utils import *
import sys
sys.path.append('..')

s_per_frame = 256 / 22050

dataset_folders = ["BACH10_processed", "URMP_processed", "PHENICX_processed"]

### 运行模型

In [2]:
model_folder_name = "basicamt_nolog"
sys.path.append(f'../basicamt/ablation/nolog/')
from basicamt_nolog import BasicAMT_all_nL as BasicAMT_all
torch.serialization.add_safe_globals([BasicAMT_all])
model = torch.load(f"../basicamt/ablation/nolog/basicamt_model.pth", weights_only=False)
model.eval()

OptimizedModule(
  (_orig_mod): BasicAMT_all_nL(
    (eng): EnergyNorm()
    (HCQT): HarmonicaStacking()
    (early_conv): Sequential(
      (0): CBLR(
        (conv): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=same, bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): LeakyReLU(negative_slope=0.01, inplace=True)
      )
      (1): Conv2d(16, 8, kernel_size=(25, 3), stride=(1, 1), padding=(36, 1), dilation=(3, 1))
      (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (res): ReLU(inplace=True)
    (neck): CBR(
      (conv): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=same, bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (conv_yn): Sequential(
      (0): Conv2d(16, 1, kernel_size=(5, 5), stride=(3, 1), padding=(1, 2))
      (1): Sigmoid()
    )
    (con

In [3]:
# 得到所有运行结果
with torch.no_grad():
    for dataset_folder in dataset_folders:
        amt_dataset(model, dataset_folder, f"./{model_folder_name}")

processing BACH10_processed
	Finish 06-DieSonne@4
	Finish 08-FuerDeinenThron@1
	Finish 08-FuerDeinenThron@0
	Finish 02-AchLiebenChristen@0
	Finish 10-NunBitten@2
	Finish 06-DieSonne@1
	Finish 05-DieNacht@0
	Finish 09-Jesus@3
	Finish 03-ChristederdubistTagundLicht@1
	Finish 08-FuerDeinenThron@3
	Finish 02-AchLiebenChristen@4
	Finish 04-ChristeDuBeistand@4
	Finish 06-DieSonne@0
	Finish 01-AchGottundHerr@2
	Finish 05-DieNacht@3
	Finish 03-ChristederdubistTagundLicht@4
	Finish 05-DieNacht@1
	Finish 05-DieNacht@2
	Finish 04-ChristeDuBeistand@0
	Finish 01-AchGottundHerr@0
	Finish 07-HerrGott@1
	Finish 09-Jesus@4
	Finish 04-ChristeDuBeistand@2
	Finish 01-AchGottundHerr@3
	Finish 04-ChristeDuBeistand@1
	Finish 07-HerrGott@3
	Finish 09-Jesus@1
	Finish 02-AchLiebenChristen@2
	Finish 10-NunBitten@3
	Finish 09-Jesus@2
	Finish 08-FuerDeinenThron@4
	Finish 01-AchGottundHerr@4
	Finish 02-AchLiebenChristen@1
	Finish 05-DieNacht@4
	Finish 07-HerrGott@0
	Finish 07-HerrGott@4
	Finish 06-DieSonne@3
	Finis

### 计算帧级指标，并寻找最好阈值

In [4]:
# 为了方便粘贴到excel
output_results = []

In [5]:
# 只看BACH10的合奏结果
npyfolder = f"{model_folder_name}/BACH10_eval"
npys = [os.path.join(npyfolder, f) for f in os.listdir(npyfolder) if f.endswith("0.npy")]
best_thre, max_acc, max_p, max_r, max_f1 = find_best_threshold(npys, (0.1, 0.5), step_num=10, generation=4, log=True)
print("note level evaluation at best threshold:")
best_onset_thre, note_p, note_r, note_f, note_overlap = find_best_onset_threshold(npys, best_thre, (0.2, 0.6), step_num=10, generation=4, log=True)

output_results.extend([best_thre, max_acc, max_p, max_r, max_f1, best_onset_thre, note_p, note_r, note_f, note_overlap])

| threshold | Acc | P | R | F1 |
| --------- | --- |---|---|----|
| 0.10000 | 0.63765 | 0.71422 | 0.85630 | 0.77853 |
| 0.14000 | 0.65663 | 0.77865 | 0.80761 | 0.79250 |
| 0.18000 | 0.65250 | 0.81959 | 0.76233 | 0.78950 |
| Best threshold | 0.14000 | ~ | ~ | F1: 0.79250 |
| 0.10727 | 0.64378 | 0.72823 | 0.84761 | 0.78308 |
| 0.11455 | 0.64790 | 0.74047 | 0.83846 | 0.78612 |
| 0.12182 | 0.65168 | 0.75263 | 0.82950 | 0.78888 |
| 0.12909 | 0.65425 | 0.76362 | 0.82064 | 0.79078 |
| 0.13636 | 0.65565 | 0.77354 | 0.81166 | 0.79178 |
| 0.14364 | 0.65681 | 0.78291 | 0.80337 | 0.79263 |
| 0.15091 | 0.65717 | 0.79176 | 0.79488 | 0.79291 |
| 0.15818 | 0.65737 | 0.79967 | 0.78736 | 0.79306 |
| 0.16545 | 0.65583 | 0.80656 | 0.77862 | 0.79193 |
| Best threshold | 0.15818 | ~ | ~ | F1: 0.79306 |
| 0.15223 | 0.65706 | 0.79301 | 0.79347 | 0.79284 |
| 0.15355 | 0.65740 | 0.79459 | 0.79238 | 0.79308 |
| 0.15488 | 0.65740 | 0.79594 | 0.79105 | 0.79308 |
| 0.15620 | 0.65716 | 0.79731 | 0.78937 | 0.79291 |


In [6]:
# 看BACH10独奏的结果
npyfolder = f"{model_folder_name}/BACH10_eval"
npys = [os.path.join(npyfolder, f) for f in os.listdir(npyfolder) if not f.endswith("0.npy")]
best_thre, max_acc, max_p, max_r, max_f1 = find_best_threshold(npys, (0.4, 0.5), step_num=10, generation=4, log=True)
print("note level evaluation at best threshold:")
best_onset_thre, note_p, note_r, note_f, note_overlap = find_best_onset_threshold(npys, best_thre, (0.2, 0.6), step_num=10, generation=4, log=True)

output_results.extend([best_thre, max_acc, max_p, max_r, max_f1, best_onset_thre, note_p, note_r, note_f, note_overlap])

| threshold | Acc | P | R | F1 |
| --------- | --- |---|---|----|
| 0.40000 | 0.81213 | 0.88548 | 0.90074 | 0.89097 |
| 0.41000 | 0.81286 | 0.88766 | 0.89931 | 0.89144 |
| 0.42000 | 0.81353 | 0.88972 | 0.89795 | 0.89186 |
| 0.43000 | 0.81407 | 0.89162 | 0.89664 | 0.89222 |
| 0.44000 | 0.81422 | 0.89318 | 0.89514 | 0.89233 |
| 0.45000 | 0.81474 | 0.89502 | 0.89386 | 0.89267 |
| 0.46000 | 0.81535 | 0.89684 | 0.89279 | 0.89308 |
| 0.47000 | 0.81536 | 0.89847 | 0.89117 | 0.89313 |
| 0.48000 | 0.81535 | 0.90004 | 0.88952 | 0.89314 |
| 0.49000 | 0.81520 | 0.90141 | 0.88786 | 0.89303 |
| Best threshold | 0.48000 | ~ | ~ | F1: 0.89314 |
| 0.47182 | 0.81526 | 0.89867 | 0.89081 | 0.89306 |
| 0.47364 | 0.81527 | 0.89894 | 0.89056 | 0.89308 |
| 0.47545 | 0.81530 | 0.89927 | 0.89025 | 0.89309 |
| 0.47727 | 0.81536 | 0.89961 | 0.88998 | 0.89314 |
| 0.47909 | 0.81545 | 0.89994 | 0.88974 | 0.89320 |
| 0.48091 | 0.81536 | 0.90018 | 0.88938 | 0.89314 |
| Best threshold | 0.47909 | ~ | ~ | F1: 0.89320 |


In [7]:
# 看PHENICX合奏音频的结果
npyfolder = f"{model_folder_name}/PHENICX_eval"
npys = [os.path.join(npyfolder, f) for f in os.listdir(npyfolder) if f.endswith(".npy")]
best_thre, max_acc, max_p, max_r, max_f1 = find_best_threshold(npys, (0.02, 0.2), step_num=10, generation=4, log=True)
print("note level evaluation at best threshold:")
best_onset_thre, note_p, note_r, note_f, note_overlap = find_best_onset_threshold(npys, best_thre, (0.1, 0.6), step_num=10, generation=4, log=True)

output_results.extend([best_thre, max_acc, max_p, max_r, max_f1, best_onset_thre, note_p, note_r, note_f, note_overlap])

| threshold | Acc | P | R | F1 |
| --------- | --- |---|---|----|
| 0.02000 | 0.28842 | 0.30746 | 0.82536 | 0.44744 |
| 0.03800 | 0.40044 | 0.46679 | 0.73957 | 0.57163 |
| 0.05600 | 0.43241 | 0.54610 | 0.67471 | 0.60316 |
| 0.07400 | 0.43912 | 0.59729 | 0.62235 | 0.60920 |
| 0.09200 | 0.43494 | 0.63328 | 0.57934 | 0.60474 |
| Best threshold | 0.07400 | ~ | ~ | F1: 0.60920 |
| 0.05927 | 0.43473 | 0.55687 | 0.66413 | 0.60534 |
| 0.06255 | 0.43634 | 0.56654 | 0.65424 | 0.60682 |
| 0.06582 | 0.43795 | 0.57625 | 0.64497 | 0.60829 |
| 0.06909 | 0.43862 | 0.58505 | 0.63549 | 0.60885 |
| 0.07236 | 0.43896 | 0.59328 | 0.62654 | 0.60908 |
| 0.07564 | 0.43912 | 0.60115 | 0.61812 | 0.60915 |
| 0.07891 | 0.43904 | 0.60863 | 0.61011 | 0.60900 |
| Best threshold | 0.07400 | ~ | ~ | F1: 0.60920 |
| 0.07132 | 0.43884 | 0.59069 | 0.62926 | 0.60900 |
| 0.07192 | 0.43890 | 0.59207 | 0.62780 | 0.60904 |
| 0.07251 | 0.43900 | 0.59368 | 0.62617 | 0.60912 |
| 0.07311 | 0.43892 | 0.59494 | 0.62458 | 0.60903 |


In [8]:
# 只看URMP的合奏结果
npyfolder = f"{model_folder_name}/URMP_eval"
npys = [os.path.join(npyfolder, f) for f in os.listdir(npyfolder) if f.endswith("0.npy")]
best_thre, max_acc, max_p, max_r, max_f1 = find_best_threshold(npys, (0.15, 0.5), step_num=10, generation=4, log=True)
print("note level evaluation at best threshold:")
best_onset_thre, note_p, note_r, note_f, note_overlap = find_best_onset_threshold(npys, best_thre, (0.18, 0.6), step_num=10, generation=4, log=True)

output_results.extend([best_thre, max_acc, max_p, max_r, max_f1, best_onset_thre, note_p, note_r, note_f, note_overlap])

| threshold | Acc | P | R | F1 |
| --------- | --- |---|---|----|
| 0.15000 | 0.57893 | 0.75152 | 0.71426 | 0.72996 |
| 0.18500 | 0.57420 | 0.78610 | 0.67912 | 0.72563 |
| Best threshold | 0.15000 | ~ | ~ | F1: 0.72996 |
| 0.12136 | 0.57466 | 0.71177 | 0.74721 | 0.72685 |
| 0.12773 | 0.57652 | 0.72183 | 0.73945 | 0.72828 |
| 0.13409 | 0.57758 | 0.73092 | 0.73183 | 0.72908 |
| 0.14045 | 0.57847 | 0.73950 | 0.72477 | 0.72973 |
| 0.14682 | 0.57884 | 0.74768 | 0.71762 | 0.72993 |
| 0.15318 | 0.57879 | 0.75511 | 0.71083 | 0.72981 |
| Best threshold | 0.15000 | ~ | ~ | F1: 0.72996 |
| 0.14479 | 0.57869 | 0.74505 | 0.71983 | 0.72984 |
| 0.14595 | 0.57881 | 0.74659 | 0.71859 | 0.72992 |
| 0.14711 | 0.57885 | 0.74805 | 0.71731 | 0.72994 |
| 0.14826 | 0.57888 | 0.74945 | 0.71608 | 0.72995 |
| 0.14942 | 0.57893 | 0.75084 | 0.71486 | 0.72997 |
| 0.15058 | 0.57889 | 0.75218 | 0.71361 | 0.72993 |
| Best threshold | 0.14942 | ~ | ~ | F1: 0.72997 |
| 0.14847 | 0.57887 | 0.74969 | 0.71584 | 0.72994 |
|

In [9]:
# 只看URMP的独奏结果
npyfolder = f"{model_folder_name}/URMP_eval"
npys = [os.path.join(npyfolder, f) for f in os.listdir(npyfolder) if not f.endswith("0.npy")]
best_thre, max_acc, max_p, max_r, max_f1 = find_best_threshold(npys, (0.15, 0.4), step_num=10, generation=4, log=True)
print("note level evaluation at best threshold:")
best_onset_thre, note_p, note_r, note_f, note_overlap = find_best_onset_threshold(npys, best_thre, (0.18, 0.6), step_num=10, generation=4, log=True)

output_results.extend([best_thre, max_acc, max_p, max_r, max_f1, best_onset_thre, note_p, note_r, note_f, note_overlap])

| threshold | Acc | P | R | F1 |
| --------- | --- |---|---|----|
| 0.15000 | 0.63693 | 0.70986 | 0.85845 | 0.77112 |
| 0.17500 | 0.65716 | 0.74286 | 0.84742 | 0.78632 |
| 0.20000 | 0.67084 | 0.76882 | 0.83681 | 0.79636 |
| 0.22500 | 0.68070 | 0.79044 | 0.82683 | 0.80347 |
| 0.25000 | 0.68717 | 0.80824 | 0.81725 | 0.80802 |
| 0.27500 | 0.69177 | 0.82354 | 0.80859 | 0.81122 |
| 0.30000 | 0.69447 | 0.83676 | 0.79997 | 0.81301 |
| 0.32500 | 0.69580 | 0.84805 | 0.79174 | 0.81379 |
| 0.35000 | 0.69651 | 0.85852 | 0.78394 | 0.81418 |
| 0.37500 | 0.69601 | 0.86779 | 0.77594 | 0.81365 |
| Best threshold | 0.35000 | ~ | ~ | F1: 0.81418 |
| 0.32955 | 0.69593 | 0.85003 | 0.79023 | 0.81385 |
| 0.33409 | 0.69604 | 0.85190 | 0.78884 | 0.81392 |
| 0.33864 | 0.69622 | 0.85384 | 0.78745 | 0.81402 |
| 0.34318 | 0.69631 | 0.85569 | 0.78601 | 0.81406 |
| 0.34773 | 0.69644 | 0.85758 | 0.78460 | 0.81414 |
| 0.35227 | 0.69652 | 0.85940 | 0.78323 | 0.81417 |
| 0.35682 | 0.69636 | 0.86104 | 0.78167 | 0.81402 |

In [10]:
# 输出参数数量
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model.cqt))
print(count_parameters(model))

0
18976


In [11]:
print('|'.join([f"{x:.5f}" for x in output_results]))

0.15488|0.65740|0.79594|0.79105|0.79308|0.26098|0.59487|0.76565|0.66813|0.73387|0.47893|0.81547|0.89993|0.88978|0.89321|0.26314|0.81069|0.87366|0.83600|0.82260|0.07400|0.43912|0.59729|0.62235|0.60920|0.13974|0.49039|0.44328|0.46150|0.39596|0.14942|0.57893|0.75084|0.71486|0.72997|0.20644|0.63547|0.69985|0.65718|0.62038|0.35000|0.69651|0.85852|0.78394|0.81418|0.22493|0.79488|0.84135|0.80857|0.69666
